In [2]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import os
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go  # Import Plotly's graph_objects

# Create a Dash web application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.run_server(debug=True, port=8058)  # Change the port to 8050 or any other available port

# Load your dataset
df = pd.read_csv('1191 preprocessed.csv')  # Replace with your dataset file

# Define CSS styles to enhance the appearance
app.layout = dbc.Tabs([
    dbc.Tab(label="Load and Preprocess", children=[
        html.Div([
            html.H1("Load and Preprocess", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-data"),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Import Pre-Processed Emails", children=[
        html.Div([
            html.H1("Import Pre-Processed Emails", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-processed-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-processed-data"),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Sentiment Classification", children=[
        html.Div([
            html.H1("Sentiment Classification", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-sentiment-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-sentiment-data"),
            dcc.Textarea(id='input-text', placeholder='Enter text for sentiment analysis...'),
            html.Div([
                html.Button('Analyze Sentiment', id='analyze-button', style={'margin-top': '10px'}),
                html.Div(id='output-sentiment'),
            ]),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Export Emails", children=[
        html.Div([
            html.H1("Export Emails", style={'color': '#0080FF', 'font-size': '36px'}),
            html.Button("Export Processed Data", id="export-processed-button", n_clicks=0),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Visualization", children=[
        html.Div([
            html.H1("Visualization", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Dropdown(
                id='visualization-dropdown',
                options=[
                    {'label': 'Word Cloud', 'value': 'word-cloud'},
                    {'label': 'Network Graph', 'value': 'network-graph'},
                    {'label': 'Time Series', 'value': 'time-series'},
                    {'label': 'Tree Map', 'value': 'tree-map'},
                    {'label': 'Pie Chart', 'value': 'pie-chart'}
                ],
                value='word-cloud'
            ),
            dcc.Graph(id='visualization-graph'),
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
])

@app.callback(
    Output("download-data", "data"),
    Input("upload-data", "filename"),
    Input("upload-data", "contents"),
)
def download_uploaded_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output("download-processed-data", "data"),
    Input("upload-processed-data", "filename"),
    Input("upload-processed-data", "contents"),
)
def download_uploaded_processed_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output("download-sentiment-data", "data"),
    Input("upload-sentiment-data", "filename"),
    Input("upload-sentiment-data", "contents"),
)
def download_uploaded_sentiment_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output('visualization-graph', 'figure'),
    Input('visualization-dropdown', 'value')
)
def update_visualization(selected_option):
    if selected_option == 'word-cloud':
        text = ' '.join(df['Content'])
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

        plt.figure(figsize=(8, 4))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.tight_layout()
        wordcloud_image_path = 'wordcloud.png'
        plt.savefig(wordcloud_image_path, bbox_inches='tight')

        return {
            'data': [],
            'layout': {
                'xaxis': {'visible': False},
                'yaxis': {'visible': False},
                'annotations': [
                    {
                        'text': 'Word Cloud',
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 28},
                    }
                ],
                'images': [
                    {
                        'source': 'data:image/png;base64,{}'.format(wordcloud_image_path),
                        'x': 0,
                        'y': 1,
                        'xref': 'paper',
                        'yref': 'paper',
                        'sizex': 1,
                        'sizey': 1,
                        'xanchor': 'left',
                        'yanchor': 'top',
                    }
                ]
            }
        }
    elif selected_option == 'tree-map':
        # Create a Tree Map
        df_2000 = df[df['Year'] == 2000]

    # Create a Tree Map for the year 2000
        tree_map_fig = px.treemap(df_2000, path=['Year', 'Label'], color='Label')
        return tree_map_fig
    
    
    elif selected_option == 'network-graph':
        # Create a Network Graph (Add your code here)
        return {}
    elif selected_option == 'time-series':
        # Create a Time Series (Add your code here)
        return {}
    elif selected_option == 'pie-chart':
        # Create a Pie Chart (Add your code here)
        return {}

@app.callback(
    Output('output-sentiment', 'children'),
    Input('analyze-button', 'n_clicks'),
    State('input-text', 'value')
)
def analyze_sentiment(n_clicks, input_text):
    # Perform sentiment analysis here and return the result
    return "Sentiment: Positive"  # Replace with your analysis result

if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
NoLayoutException                         Traceback (most recent call last)
File ~/anaconda3/lib/python3.11/site-packages/flask/app.py:1818, in Flask.full_dispatch_request(self=<Flask '__main__'>)
   1816 try:
   1817     request_started.send(self)
-> 1818     rv = self.preprocess_request()
        self = <Flask '__main__'>
   1819     if rv is None:
   1820         rv = self.dispatch_request()

File ~/anaconda3/lib/python3.11/site-packages/flask/app.py:2309, in Flask.preprocess_request(self=<Flask '__main__'>)
   2307 if name in self.before_request_funcs:
   2308     for before_func in self.before_request_funcs[name]:
-> 2309         rv = self.ensure_sync(before_func)()
        before_func = <bound method Dash._setup_server of <dash.dash.Dash object at 0x11275b2d0>>
        self = <Flask '__main__'>
   2311         if rv is not None:
   2312             return rv

File ~/anaconda3/lib/python3.11/site-packages/

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: sequence item 110: expected str instance, float found

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: sequence item 110: expected str instance, float found

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: sequence item 110: expected str instance, float found



In [6]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go

# Create a Dash web application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Load your dataset
df = pd.read_csv('1191 preprocessed.csv')  # Replace with your dataset file

app.run_server(debug=True, port=8058)  # Change the port to 8050 or any other available port

# Define CSS styles to enhance the appearance
app.layout = dbc.Tabs([
    dbc.Tab(label="Load and Preprocess", children=[
        html.Div([
            html.H1("Load and Preprocess", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-data"),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Import Pre-Processed Emails", children=[
        html.Div([
            html.H1("Import Pre-Processed Emails", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-processed-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-processed-data"),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Sentiment Classification", children=[
        html.Div([
            html.H1("Sentiment Classification", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-sentiment-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-sentiment-data"),
            dcc.Textarea(id='input-text', placeholder='Enter text for sentiment analysis...'),
            html.Div([
                html.Button('Analyze Sentiment', id='analyze-button', style={'margin-top': '10px'}),
                html.Div(id='output-sentiment'),
            ]),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Export Emails", children=[
        html.Div([
            html.H1("Export Emails", style={'color': '#0080FF', 'font-size': '36px'}),
            html.Button("Export Processed Data", id="export-processed-button", n_clicks=0),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Visualization", children=[
        html.Div([
            html.H1("Visualization", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Dropdown(
                id='visualization-dropdown',
                options=[
                    {'label': 'Word Cloud', 'value': 'word-cloud'},
                    {'label': 'Network Graph', 'value': 'network-graph'},
                    {'label': 'Time Series', 'value': 'time-series'},
                    {'label': 'Tree Map', 'value': 'tree-map'},
                    {'label': 'Pie Chart', 'value': 'pie-chart'}
                ],
                value='word-cloud'
            ),
            dcc.Graph(id='visualization-graph'),
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
])

@app.callback(
    Output("download-data", "data"),
    Input("upload-data", "filename"),
    Input("upload-data", "contents"),
)
def download_uploaded_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output("download-processed-data", "data"),
    Input("upload-processed-data", "filename"),
    Input("upload-processed-data", "contents"),
)
def download_uploaded_processed_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output("download-sentiment-data", "data"),
    Input("upload-sentiment-data", "filename"),
    Input("upload-sentiment-data", "contents"),
)
def download_uploaded_sentiment_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output('visualization-graph', 'figure'),
    Input('visualization-dropdown', 'value')
)
def update_visualization(selected_option):
    if selected_option == 'word-cloud':
        text = ' '.join(df['Content'])
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

        plt.figure(figsize=(8, 4))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.tight_layout()
        wordcloud_image_path = 'wordcloud.png'
        plt.savefig(wordcloud_image_path, bbox_inches='tight')

        return {
            'data': [],
            'layout': {
                'xaxis': {'visible': False},
                'yaxis': {'visible': False},
                'annotations': [
                    {
                        'text': 'Word Cloud',
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 28},
                    }
                ],
                'images': [
                    {
                        'source': 'data:image/png;base64,{}'.format(wordcloud_image_path),
                        'x': 0,
                        'y': 1,
                        'xref': 'paper',
                        'yref': 'paper',
                        'sizex': 1,
                        'sizey': 1,
                        'xanchor': 'left',
                        'yanchor': 'top',
                    }
                ]
            }
        }
    elif selected_option == 'tree-map':
        # Create a Tree Map for the year 2000
        df_2000 = df[df['Year'] == 2000]
        tree_map_fig = px.treemap(df_2000, path=['Year', 'Label'], color='Label')
        return tree_map_fig
    elif selected_option == 'network-graph':
        # Create a Network Graph (Add your code here)
        return {}
    elif selected_option == 'time-series':
        # Create a Time Series (Add your code here)
        return {}
    elif selected_option == 'pie-chart':
        # Create a Pie Chart (Add your code here)
        return {}

@app.callback(
    Output('output-sentiment', 'children'),
    Input('analyze-button', 'n_clicks'),
    State('input-text', 'value')
)
def analyze_sentiment(n_clicks, input_text):
    # Perform sentiment analysis here and return the result
    return "Sentiment: Positive"  # Replace with your analysis result

if __name__ == '__main__':
    app.run_server(port=8058)


---------------------------------------------------------------------------
NoLayoutException                         Traceback (most recent call last)
File ~/anaconda3/lib/python3.11/site-packages/flask/app.py:1818, in Flask.full_dispatch_request(self=<Flask '__main__'>)
   1816 try:
   1817     request_started.send(self)
-> 1818     rv = self.preprocess_request()
        self = <Flask '__main__'>
   1819     if rv is None:
   1820         rv = self.dispatch_request()

File ~/anaconda3/lib/python3.11/site-packages/flask/app.py:2309, in Flask.preprocess_request(self=<Flask '__main__'>)
   2307 if name in self.before_request_funcs:
   2308     for before_func in self.before_request_funcs[name]:
-> 2309         rv = self.ensure_sync(before_func)()
        before_func = <bound method Dash._setup_server of <dash.dash.Dash object at 0x145367f50>>
        self = <Flask '__main__'>
   2311         if rv is not None:
   2312             return rv

File ~/anaconda3/lib/python3.11/site-packages/

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: sequence item 110: expected str instance, float found

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: sequence item 110: expected str instance, float found



In [8]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go

# Create a Dash web application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define CSS styles to enhance the appearance
app.layout = dbc.Tabs([
    dbc.Tab(label="Load and Preprocess", children=[
        html.Div([
            html.H1("Load and Preprocess", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-data"),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Import Pre-Processed Emails", children=[
        html.Div([
            html.H1("Import Pre-Processed Emails", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-processed-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-processed-data"),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Sentiment Classification", children=[
        html.Div([
            html.H1("Sentiment Classification", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Upload(
                id='upload-sentiment-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                }
            ),
            dcc.Download(id="download-sentiment-data"),
            dcc.Textarea(id='input-text', placeholder='Enter text for sentiment analysis...'),
            html.Div([
                html.Button('Analyze Sentiment', id='analyze-button', style={'margin-top': '10px'}),
                html.Div(id='output-sentiment'),
            ]),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Export Emails", children=[
        html.Div([
            html.H1("Export Emails", style={'color': '#0080FF', 'font-size': '36px'}),
            html.Button("Export Processed Data", id="export-processed-button", n_clicks=0),
            # Add more content for this scenario
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
    dbc.Tab(label="Visualization", children=[
        html.Div([
            html.H1("Visualization", style={'color': '#0080FF', 'font-size': '36px'}),
            dcc.Dropdown(
                id='visualization-dropdown',
                options=[
                    {'label': 'Word Cloud', 'value': 'word-cloud'},
                    {'label': 'Network Graph', 'value': 'network-graph'},
                    {'label': 'Time Series', 'value': 'time-series'},
                    {'label': 'Tree Map', 'value': 'tree-map'},
                    {'label': 'Pie Chart', 'value': 'pie-chart'}
                ],
                value='word-cloud'
            ),
            dcc.Graph(id='visualization-graph'),
        ], className="tab-content", style={'background-color': '#EFFBFB', 'padding': '20px', 'border-radius': '10px'}),
    ]),
])

# Load your dataset
df = pd.read_csv('1191 preprocessed.csv')  # Replace with your dataset file

@app.callback(
    Output("download-data", "data"),
    Input("upload-data", "filename"),
    Input("upload-data", "contents"),
)
def download_uploaded_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output("download-processed-data", "data"),
    Input("upload-processed-data", "filename"),
    Input("upload-processed-data", "contents"),
)
def download_uploaded_processed_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output("download-sentiment-data", "data"),
    Input("upload-sentiment-data", "filename"),
    Input("upload-sentiment-data", "contents"),
)
def download_uploaded_sentiment_data(filename, contents):
    if filename is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return {"content": decoded, "filename": filename}

@app.callback(
    Output('visualization-graph', 'figure'),
    Input('visualization-dropdown', 'value')
)
def update_visualization(selected_option):
    if selected_option == 'word-cloud':
        # Convert the "Content" column to strings and handle NaN values
        text = ' '.join(df['Content'].astype(str).fillna(''))
        
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

        plt.figure(figsize=(8, 4))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.tight_layout()
        wordcloud_image_path = 'wordcloud.png'
        plt.savefig(wordcloud_image_path, bbox_inches='tight')

        return {
            'data': [],
            'layout': {
                'xaxis': {'visible': False},
                'yaxis': {'visible': False},
                'annotations': [
                    {
                        'text': 'Word Cloud',
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 28},
                    }
                ],
                'images': [
                    {
                        'source': 'data:image/png;base64,{}'.format(wordcloud_image_path),
                        'x': 0,
                        'y': 1,
                        'xref': 'paper',
                        'yref': 'paper',
                        'sizex': 1,
                        'sizey': 1,
                        'xanchor': 'left',
                        'yanchor': 'top',
                    }
                ]
            }
        }
    elif selected_option == 'tree-map':
        # Create a Tree Map for the year 2000
        df_2000 = df[df['Year'] == 2000]
        tree_map_fig = px.treemap(df_2000, path=['Year', 'Label'], color='Label')
        return tree_map_fig
    elif selected_option == 'network-graph':
        # Create a Network Graph (Add your code here)
        return {}
    elif selected_option == 'time-series':
        # Create a Time Series (Add your code here)
        return {}
    elif selected_option == 'pie-chart':
        # Create a Pie Chart (Add your code here)
        return {}


@app.callback(
    Output('output-sentiment', 'children'),
    Input('analyze-button', 'n_clicks'),
    State('input-text', 'value')
)
def analyze_sentiment(n_clicks, input_text):
    # Perform sentiment analysis here and return the result
    return "Sentiment: Positive"  # Replace with your analysis result

if __name__ == '__main__':
    app.run_server(port=8058)
